#

# The Basics

The chapter will introduce some bare basics.  In order to explain some things, we will need to briefly go over how
lifetimes work.  It's assumed you already followed Chapter 0, the prerequisites.

In [ ]:
use std::fmt::{Display, Formatter};
use reqwest::{Client, Response, Error};


/// 
pub fn pipe<A, B, C>(f1: impl Fn(A) -> B, f2: impl Fn(B) -> C) -> impl Fn(A) -> C {
    move |a: A| {
        f2(f1(a))
    }
}


pub fn move_example() {
    let x = 10;
    let y = x;
    println!("x is {}, y is {}", x, y);

    let name = String::from("Sean");
    let copied = &name;
    println!("name  is {}, copied is {}", name, copied);
}

pub async fn post_request(json: String, token: &str) -> Result<Response,Error>  {
    let client = Client::new();
     client.post("https:/my.cool.service/v2/awesome/endpoint")
        .body(json)
        .header("Content-Type", "application/json")
        .header("Accept", "application/json")
        .header("Bearer ", token)
        .send()
        .await
}

pub async fn determine_response(response: Result<Response, Error>) -> Option<String> {
    match response {
        Ok(resp) => {
            if resp.status().is_success() { resp.text().await.ok() } else { None } 
        },
        Err(e) => Some(format!("Got error: {}", e))
    }
}

pub fn concat(left: &str, right: &str) -> String { // a &str is a view into preallocated memory, String lives on the heap
    format!("{}{}", left, right)
}

pub fn mut_concat(left: &mut String, right: &str) { // the &mut says, mutable reference, so ownership is not transferred
    left.push_str(right) // mutate the data that String points to by adding `right` to the contigous memory
}

pub fn test_concat() {
    let name = "sean";  // Has type of &str
    let new = concat(name, " toner");
    println!("name is {}, new name is {}", name, new);  // name is sean, new name is sean toner

    let mut left = String::from("sean");
    mut_concat(&mut left, " toner");
    println!("left is {}", left) // left  is sean toner
}

pub fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}



pub struct Engineer<'a> {
    pub name: &'a str,
    pub languages: Vec<&'a str>,
    pub experience: u32
}

impl<'a> Engineer<'a> {
    pub fn lang_str(&self) -> String {
        format!("[{}]", self.languages.iter()
            .map(|lang| lang.to_owned())
            .collect::<Vec<&'a str>>()
            .join(","))
    }
}

impl<'a> Learn for Engineer<'a> {
    type Arg = &'a str;

    fn add_skill(&mut self, skill: Self::Arg) {
        self.languages.push(skill);
    }
}

pub struct EngineerOwned {
    pub name: String,
    pub languages: Vec<String>,
    pub experience: u32
}

impl EngineerOwned {
    pub fn lang_str(&self) -> String {
        format!("[{}]", self.languages.iter()
            .map(|lang| lang.to_owned())
            .collect::<Vec<String>>()
            .join(","))
    }
}

trait Learn {
    type Arg;

    fn add_skill(&mut self, skill: Self::Arg);
}

impl Learn for EngineerOwned {
    type Arg = String;

    fn add_skill(&mut self, skill: Self::Arg) {
        self.languages.push(skill);
    }
}

impl<'a> Display for Engineer<'a> {
    fn fmt(&self, f: &mut Formatter<'_>) -> std::fmt::Result {
        write!(f, r#"{{ 
            "name": "{}",
            "languages": "{}",
            "experience", "{}"
        }}"#, self.name, self.lang_str(), self.experience)
    }
}

pub fn test() {
    let mut eng = Engineer { name: "Sean", experience: 16, languages: vec!["rust", "java", "kotlin"] };
    eng.add_skill("scala3");
    println!("Engineer's name is {}", eng.name);
    println!("Engineer's experience is {} years", eng.experience);
    println!("Engineer's known languages are {}", eng.lang_str());
    eng.languages.first();
    println!("{}", eng);
}

pub struct Date {
    pub year: u32,
    pub month: u32,
    pub date: u32
}

pub enum DateTypes {
    DateString(String),
    AsDate(Date)
}

use crate::DateTypes::{DateString, AsDate};
pub fn date_to_string(dt: DateTypes) -> String {
    match dt {
        DateString(date) => format!("The date is {}", date),
        AsDate(Date {year: y, month: m, date: d}) => format!("The date is {}-{}-{}", y, m, d)
    }
}
